In [30]:
#
# NTRU
#

# Implementation based on the book
# "Cryptography: Theory and Practice" -
# Stinson and Paterson

# Author: Laura Viglioni
# Licence: GPL-v3
# 2020

In [31]:
#
# imports
#

from collections import namedtuple
from functools import reduce
import datetime

In [32]:
#
# Define some functions to be used later
#

# Compare two polynomials in a ring
def equals_poly(p1,p2, ring):
    c1 = list(ring(p1))
    c2 = list(ring(p2))
    equals = list(map(lambda x,y: x==y, c1, c2))
    return reduce(lambda acc,val: acc and val, equals, true )


# Returns a function that calculates a mods
def mods_hof(modulus):
    def _mods(number): 
        reduced = mod(number, modulus)
        if (reduced > floor(modulus/2)):
              return Integer(reduced) - Integer(modulus)
        else:
            return reduced
    return _mods


# Returns a random (-1,0 or 1)
def rnd_coef(_):
    return int(round(random()*2-1))


# define random coef function
def rnd_coefs(degree):
    return list(map(rnd_coef, range(degree+1)))

In [40]:
#
# Define a instance of NTRUE
#

# NTRUE.keys(): generate and returns public_key and private_key 
# NTRUE.encrypt(message, public_key): encrypts a message using pubk
# NTRUE.decrypt(encrypted, private_key): decrypts a message using prik

# NOTES: 
# N and q must be a prime number
# p must be odd
# For values of N > 10000 this alg may not work

def NTRUE(p,q,N):
    # ring defs
    ring = PolynomialRing(ZZ, 'x')
    _ring = PolynomialRing(Integers(q), 'x')
    modulus = x^N-1
    qring = _ring.quotient(modulus, 'x')
    
    # define mods function
    mods = mods_hof(q)
    def _mods_list(l, mods_func=mods):
        return list(map(mods_func, l))   
    
    # multiply two polynomials mods x^N-1 
    def mul_poly(p1, p2):
        res = (qring(p1)*qring(p2)).mod(modulus)
        coefs = _mods_list(list(res))
        return ring(coefs)
    
    # multiply a number to a polynomial 
    def scalar_mul(poly, number):
        coefs = ring(poly).list()
        multiplied = list(map(lambda c: mods(number*c) , coefs))
        return ring(multiplied)
    
    # sum two polys
    def sum_poly(p1,p2):
        res = (qring(p1) + qring(p2)).mod(modulus)
        coefs = _mods_list(list(res))
        return ring(coefs)
    
    # get random polynomial with degree N
    def rnd_poly(degree=N):
        coefs = rnd_coefs(degree)
        if abs(sum(coefs)) < 5:
            return ring(coefs)
        else:
            return rnd_poly(degree)
        
    def inverse(poly):
        inv = 1/qring(poly)
        coefs = _mods_list(list(inv))
        return ring(coefs)
        
    # define f and g from F and G
    def get_f(_F=None):
        F = _F or rnd_poly()
        return scalar_mul(F,p) + 1
    
    def get_g(_G=None):
        G = _G or rnd_poly()
        return scalar_mul(G,p)
    
    # get h from f and g
    def get_h(f,g):
        f_inv = inverse(f)
        return mul_poly(f_inv, g)
    
    # keys
    def keys(F=None,G=None):
        f = get_f(F)   # private key
        g = get_g(G)
        h = get_h(f,g) # public key
        return (h, f)
    
    # Encryption
    def encrypt(m,h,_r=None):
        r = _r or rnd_poly()
        return sum_poly(mul_poly(r,h), m)

    
    # Decryption
    def decrypt(y, f):
        a = mul_poly(y,f)
        mods_p = mods_hof(p)
        coefs = _mods_list(list(a), mods_p)
        return ring(coefs)
        
    
    ntrue = namedtuple("NTRUE", ["ring", "mul_poly", "mods", "rnd_poly", "scalar_mul", "inverse", "get_f", "get_g", "get_h", "sum_poly", "encrypt", "decrypt", "keys"])
    
    return ntrue(ring, mul_poly, mods, rnd_poly, scalar_mul, inverse, get_f, get_g, get_h, sum_poly, encrypt, decrypt, keys)



In [39]:
#
# Running example 9.1 from the book
#

# parameters:
p=3
q=31
N=23

# instance of NTRUEncrypt
ntrue = NTRUE(p,q,N)

#Chosen polynomials
F = x^18-x^9+x^8-x^4-x^2
G = x^17+x^12+x^9+x^3-x

# Message
m = x^15-x^12+x^7-1

# Chosen error polynomial
r = x^19+x^10+x^6-x^2

public_key, private_key = ntrue.keys(F,G)

# Encrypting and decrypting a message m
y = ntrue.encrypt(m,public_key,r)
m_line = ntrue.decrypt(y, private_key)

print("private key (f) is ")
print(private_key)

print("\npublic key (h) is")
print(public_key)

print("\nmessage (m) is")
print(m)

print("\nencrypted message (y) is")
print(y)

print("\ndecrypted message (m') is")
print(m_line)

print("\nThe message was correctly decrypted?")
print(equals_poly(m, m_line, ntrue.ring))



private key (f) is 
3*x^18 - 3*x^9 + 3*x^8 - 3*x^4 - 3*x^2 + 1

public key (h) is
0

message (m) is
x^15 - x^12 + x^7 - 1

encrypted message (y) is
x^15 + 2*x^12 + x^7 + 2

decrypted message (m') is
x^15 - x^12 + x^7 - 1

The message was correctly decrypted?
True


In [35]:
#
# Defining a random example
#


def rnd_example(p,q,N):
    # time measurement
    start_time = datetime.datetime.now()

    # instance of NTRUEncrypt
    ntrue = NTRUE(p,q,N)

    # get keys
    pub_k, pri_k = ntrue.keys()

    # Generate a random message (N-1 degree polynomial):
    m = ntrue.rnd_poly(N-1)

    # Encryption:
    y = ntrue.encrypt(m, pub_k)

    # Decryption:
    m_line = ntrue.decrypt(y, pri_k)

    # time measurement
    total_time = (datetime.datetime.now() - start_time).total_seconds()
    
    # Since the message m and encrypted y are too long to print
    # we will only print if m == m'
    print("\nThe message was correctly decrypted?")
    print(equals_poly(m, m_line, ntrue.ring))

    print(f'This call took {total_time} seconds using polynomials with degree {N}\n')
    

In [36]:
#
# Running random examples
#

# parameters
p=3
q= 2053

N= 401 
rnd_example(p,q,N)

N=6959
rnd_example(p,q,N)

N=9929
rnd_example(p,q,N)




The message was correctly decrypted?
True
This call took 0.185871 seconds using polynomials with degree 401


The message was correctly decrypted?
True
This call took 2.9019 seconds using polynomials with degree 6959


The message was correctly decrypted?
True
This call took 4.179865 seconds using polynomials with degree 9929



In [37]:
# parameters:
p=3
q=31
N=23

# instance of NTRUEncrypt
ntrue = NTRUE(p,q,N)

pub_k, pri_k = ntrue.keys()

m = x^2 + x

y = ntrue.encrypt(m, pub_k)

m_line = ntrue.decrypt(y, pri_k)
print(m)
print(y)
print(m_line)

pri_k

x^2 + x
14*x^22 - 5*x^21 - 13*x^20 + 9*x^19 + 9*x^18 - 6*x^17 - 10*x^16 + 15*x^15 - 10*x^14 + 15*x^13 + 8*x^12 - 6*x^11 - 14*x^10 + 6*x^9 + 13*x^8 + 14*x^7 + 7*x^5 + 4*x^4 - 9*x^2 - 15*x - 11
x^16 - x^9 + x^3 + x^2 + x


3*x^23 - 3*x^19 + 3*x^18 - 3*x^15 - 3*x^14 - 3*x^13 - 3*x^10 - 3*x^7 + 3*x^6 - 3*x^2 - 3*x + 4